In [18]:
# !pip install mwparserfromhell
# !pip install pymongo
# !pip install polyglot
# !apt-get install -y libicu-dev
# !pip install pyicu
# !pip install nltk
# !pip install pycld2
# !pip install morfessor
# !pip install pyahocorasick
# !apt install -y mongodb-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mongodb-server is already the newest version (1:3.6.3-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 119 not upgraded.


In [ ]:
from wikipedia_cleaner import PTwikitextCleaner
from wikipedia_builder import WikipediaParser
from utils import entity_iterator, clean_wikitext, sentences_builder
import pymongo

In [22]:
# !service mongodb start
# http://www.techpaste.com/2016/04/limit-mongodb-memory-usage/
# !cgcreate -g memory:MongoLimitGroup
# !echo 6000000000 > /sys/fs/cgroup/memory/MongoLimitGroup/memory.limit_in_bytes
# !sync; echo 3 > /proc/sys/vm/drop_caches
# !cgclassify -g memory:MongoLimitGroup mongod_pid

 * Starting database mongodb
   ...done.


cgcreate -g memory:MongoLimitGroup && echo 6000000000 > /sys/fs/cgroup/memory/MongoLimitGroup/memory.limit_in_bytes && sync; echo 3 > /proc/sys/vm/drop_caches && cgclassify -g memory:MongoLimitGroup 23096

In [33]:
client = pymongo.MongoClient('localhost', 27017)
client.drop_database('wikipedia_ner_corpus')

In [30]:
wiki_dump_pages = "wiki/enwiki-latest-pages-articles.xml"
mongo_host = 'localhost'
mongo_port = 27017
db_name = 'wikipedia_ner_corpus'
mappings = {'wiki_page':'wikipedia_article',
                  'tag':'tag',
                  'entity':'entity',
                  'entity_references':'entity_references'}

wiki_parser = WikipediaParser(wiki_dump_pages, mongo_host, mongo_port, db_name)

article = True
entity = True
reference = True
clean = True
build_sentences = True

In [99]:
if article:
    wiki_parser.add_dump_wikipedia_articles(bulk_size = 10000, verbose = True)

Adding to wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia_article
Adding 10000 documets in wikipedia_ner_corpus.wikipedia

AutoReconnect: localhost:27017: [WinError 10054] An existing connection was forcibly closed by the remote host

Traceback (most recent call last):
  File "d:\pycharmprojects\wiki_ner_annotation\venv\lib\site-packages\pymongo\pool.py", line 1180, in connect
    sock = _configured_socket(self.address, self.opts)
  File "d:\pycharmprojects\wiki_ner_annotation\venv\lib\site-packages\pymongo\pool.py", line 988, in _configured_socket
    sock = _create_connection(address, options)
  File "d:\pycharmprojects\wiki_ner_annotation\venv\lib\site-packages\pymongo\pool.py", line 972, in _create_connection
    raise err
  File "d:\pycharmprojects\wiki_ner_annotation\venv\lib\site-packages\pymongo\pool.py", line 965, in _create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\pycharmprojects\wiki_ner_annotation\venv\lib\site-packages\pymongo\mongo_client.py", line 1752, in _process_periodic_tasks
  

In [87]:
client['wikipedia_ner_corpus']['wikipedia_article'].estimated_document_count()

19410

In [88]:
if entity:
    wiki_parser.add_entities(entity_iterator(), bulk_size = 100, verbose = True, no_article_ignore = False)

Indexing wikipedia_ner_corpus.entity.iri
Indexing wikipedia_ner_corpus.entity.article


In [95]:
client['wikipedia_ner_corpus']['entity'].find_one()

In [93]:
tag = client['wikipedia_ner_corpus']['entity'].find_one()['tag']

TypeError: 'NoneType' object is not subscriptable

In [10]:
#wiki_parser.copy_DB('wikipedia_ner_corpus')

In [22]:
if reference:
    wiki_parser.add_references(verbose=True, bulk_size = 100000)

In [12]:
#client['wikipedia_ner_corpus']['wikipedia_article'].find_one()

In [23]:
if clean:
    wiki_parser.clean_wikitext(clean_wikitext ,bulk_size = 100000, verbose = True)

In [14]:
#for article in client['wikipedia_ner_corpus']['wikipedia_article'].find():
#    print(article['reference_count'],'---',article['title'])

In [15]:
#for r in client['wikipedia_ner_corpus']['entity_references'].find():
#    article = client['wikipedia_ner_corpus']['wikipedia_article'].find_one({'_id':r['article']})
#    entity = client['wikipedia_ner_corpus']['entity'].find_one({'_id':r['entity']})
#    print(article['title'],'-',r['text'],'->',entity['iri'])

In [16]:
#art_id = client['wikipedia_ner_corpus']['entity_references'].find_one()['article']
#print(art_id)
#len(list(client['wikipedia_ner_corpus']['entity_references'].find({'article':art_id})))

In [27]:
if build_sentences:
    wiki_parser.add_articles_sentences(sentences_builder, verbose=True, bulk_size = 100000)

In [26]:
i = 0
for sentence in client['wikipedia_ner_corpus']['wikipedia_article'].find()[4]['sentences']:
    i+=1
    print('---')
    for token in sentence['tokens']:
        print(token[0],token[1],token[2])


IndexError: no such item for Cursor instance

In [19]:
print(i)

20


In [20]:
#_id = list(client['wikipedia_ner_corpus']['wikipedia_article'].find({'title':'América Latina'}))
#print(_id)

In [21]:
#print(_id['clean_wiki_text'])

In [22]:
#client['wikipedia_ner_corpus']['wikipedia_article'].update_one({'_id':_id}, {'$set':{'sentences':None}})

In [23]:
i = 0
for sentence in list(client['wikipedia_ner_corpus']['wikipedia_article'].find({'title':'América Latina'}))[0]['sentences']:
    i+=1
    print('\n\n')
    for token in sentence['tokens']:
        print(token[0],token[1],token[2])





A O word
América B-LOC annotated
Latina I-LOC annotated
( O word
América B-LOC annotated
Latina I-LOC annotated
ou O word
Latinoamérica O word
; O word
Amérique O word
latine O word
) O word
é O word
uma O word
região O word
do O word
continente O word
americano O word
que O word
engloba O word
os O word
países O word
onde O word
são O word
faladas O word
, O word
primordialmente O word
, O word
línguas O word
românicas O word
( O word
derivadas O word
do O word
latim O word
) O word
— O word
no O word
caso O word
, O word
o O word
espanhol O word
, O word
o O word
português O word
e O word
o O word
francês O word
— O word
visto O word
que O word
, O word
historicamente O word
, O word
a O word
região O word
foi O word
maioritariamente O word
dominada O word
pelos O word
impérios O word
coloniais O word
europeus O word
Espanhol O word
e O word
Português O word
. O word



A O word
América B-LOC annotated
Latina I-LOC annotated
tem O word
uma O word
área O word
de O word
cerca O word

In [24]:
print(i)

272
